## DNN
### with pretraining
*jul 12, 2018 *

### imports

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import math

import scipy.io as sio
import numpy as np
import h5py
import tensorflow as tf
import sys

import time

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from keras.models import Model, load_model
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, LSTM, GRU, Dense, Dropout, Bidirectional
from keras.models import Sequential
from keras.layers import Dense, Activation, TimeDistributed
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint

import keras.backend as K

import matplotlib.pyplot as plt

Using TensorFlow backend.


### Variables
- Read parameters from .mat files
- save in Opt object

In [2]:
Data_VERSION = '_e10v5'
Code_VERSION = '_e03'

In [3]:
DNN_DATA_FILE = "./dnn_models/DNN_datas"+Data_VERSION+".mat"

DNN_MODEL_FILE = "./dnn_models/DNN_params"+Data_VERSION+".mat"

DNN_NET_FILE = "./dnn_models/DNN_net"+Code_VERSION+".mat"

MODEL_FILE = "./dnn_models/weights_{epoch:02d}"+ Code_VERSION+".h5"

SAVE_MODEL_FILE = "./dnn_models/py_model"+ Code_VERSION+".h5"

### Object class

In [4]:
class Opts:
    opts_dict = dict()

    def __init__(self, FILE, FILE_DATA):
        
        # Basic parameters
        with h5py.File(FILE, 'r') as f:
            key_list = list(f.keys())
            print('Opt key:',key_list)

            for k, v in f['opts'].items():

                print('key:', k)

                if k == 'ARMA_order':
                    self.ARMA_order = int(np.array(v)[0][0])
                    self.opts_dict[k] = self.ARMA_order
                elif k == 'ada_grad_eps':
                    self.ada_grad_eps = np.array(v)[0][0]
                    self.opts_dict[k] = self.ada_grad_eps
                elif k == 'ada_sgd_scale':
                    self.ada_sgd_scale = np.array(v)[0][0]
                    self.opts_dict[k] = self.ada_sgd_scale
                elif k == 'change_momentum_point':
                    self.change_momentum_point = int(np.array(v)[0][0])
                    self.opts_dict[k] = self.change_momentum_point
                elif k == 'cost_function':
                    self.cost_function = ""
                    for c in np.array(v):
                        self.cost_function += chr(c[0])

                    self.opts_dict[k] = self.cost_function

                elif k == 'cv_interval':
                    self.cv_interval = int(np.array(v)[0][0])
                    self.opts_dict[k] = self.cv_interval
                elif k == 'dim_input':
                    self.dim_input = int(np.array(v)[0][0])
                    self.opts_dict[k] = self.dim_input
                elif k == 'dim_output':
                    self.dim_output = int(np.array(v)[0][0])
                    self.opts_dict[k] = self.dim_output
                elif k == 'drop_ratio':
                    self.drop_ratio = np.array(v)[0][0]
                    self.opts_dict[k] = self.drop_ratio
                elif k == 'eval_on_gpu':
                    self.eval_on_gpu = int(np.array(v)[0][0])
                    self.opts_dict[k] = self.eval_on_gpu
                elif k == 'final_momentum':
                    self.final_momentum = int(np.array(v)[0][0])
                    self.opts_dict[k] = self.final_momentum
                elif k == 'hid_struct':
                    self.hid_struct = np.array(v)
                    self.opts_dict[k] = self.hid_struct
                elif k == 'initial_momentum':
                    self.initial_momentum = np.array(v)[0][0]
                    self.opts_dict[k] = self.initial_momentum
                elif k == 'isDropout':
                    self.isDropout = 0
                    self.opts_dict[k] = self.isDropout
                elif k == 'isDropoutInput':
                    self.isDropoutInput = int(np.array(v)[0][0])
                    self.opts_dict[k] = self.isDropoutInput
                elif k == 'isGPU':
                    self.isGPU = int(np.array(v)[0][0])
                    self.opts_dict[k] = self.isGPU
                elif k == 'isNormalize':
                    self.isNormalize = int(np.array(v)[0][0])
                    self.opts_dict[k] = self.isNormalize
                elif k == 'isPretrain':
                    self.isPretrain = int(np.array(v)[0][0])
                    self.opts_dict[k] = self.isPretrain
                elif k == 'learner':
                    self.learner = ""
                    for c in np.array(v):
                        self.learner += chr(c[0])

                    self.opts_dict[k] = self.learner

                elif k == 'net_struct':
                    self.net_struct = np.array(v)
                    for n_s in np.array(v):
                        print('Opts Net Stuct:',n_s[0])

                    self.opts_dict[k] = self.net_struct
                elif k == 'rbm_batch_size':
                    self.rbm_batch_size = int(np.array(v)[0][0])
                    print("self.rbm_batch_size:",self.rbm_batch_size)
                    # self.opts_dict[k] = self.rbm_batch_size
                elif k == 'rbm_learn_rate_binary':
                    self.rbm_learn_rate_binary = np.array(v)
                    # self.opts_dict[k] = self.rbm_learn_rate_binary
                elif k == 'rbm_learn_rate_real':
                    self.rbm_learn_rate_real = int(np.array(v)[0][0])
                    # self.opts_dict[k] = self.rbm_learn_rate_real
                elif k == 'rbm_max_epoch':
                    self.rbm_max_epoch = int(np.array(v)[0][0])
                    # self.opts_dict[k] = self.rbm_max_epoch
                elif k == 'save_on_fly':
                    self.save_on_fly = int(np.array(v)[0][0])
                    # self.opts_dict[k] = self.save_on_fly
                elif k == 'sgd_batch_size':
                    self.sgd_batch_size = int(np.array(v)[0][0])
                    print("self.sgd_batch_size:",self.sgd_batch_size)
#                     self.sgd_batch_size = 1024//2 # BATCH_SIZE for training net
                    # self.opts_dict[k] = self.sgd_batch_size
                elif k == 'sgd_learn_rate':
                    self.sgd_learn_rate = np.array(v)
                    # self.opts_dict[k] = self.sgd_learn_rate
                elif k == 'sgd_max_epoch':
                    self.sgd_max_epoch = int(np.array(v)[0][0])
                    # self.opts_dict[k] = self.sgd_max_epoch
                elif k == 'split_tanh1_c1':
                    self.split_tanh1_c1 = int(np.array(v)[0][0])
                    # self.opts_dict[k] = self.split_tanh1_c1
                elif k == 'split_tanh1_c2':
                    self.split_tanh1_c2 = int(np.array(v)[0][0])
                    # self.opts_dict[k] = self.split_tanh1_c2
                elif k == 'unit_type_hidden':
                    self.unit_type_hidden = ""
                    for c in np.array(v):
                        self.unit_type_hidden += chr(c[0])

                elif k == 'unit_type_output':
                    self.unit_type_output = ""
                    for c in np.array(v):
                        self.unit_type_output += chr(c[0])

        # Training and Dev Data 
        with h5py.File(FILE_DATA, 'r') as f:
            print('Opts h5py keys:', list(f.keys()))
            for k, v in f.items():

                if k == 'trData':
                    print("trData.shape: ", v.shape)
                    self.trData = np.transpose(np.array(v))
                    print("trData-> mean:", np.mean(self.trData), ", var:", np.var(self.trData), ", std:",
                          np.std(self.trData), ", range:", (np.amin(self.trData),np.amax(self.trData)))
                elif k == 'trLabel_i':
                    print("trLabel_i.shape: ", v.shape)
                    self.trLabel_i = np.transpose(np.array(v))
                    print("trLabel_i-> mean:", np.mean(self.trLabel_i), ", var:", np.var(self.trLabel_i), ", std:",
                          np.std(self.trLabel_i), ", range:", (np.amin(self.trLabel_i),np.amax(self.trLabel_i)))
                elif k == 'trLabel_r':
                    print("trLabel_r.shape: ", v.shape)
                    self.trLabel_r = np.transpose(np.array(v))
                    print("trLabel_r-> mean:", np.mean(self.trLabel_r), ", var:", np.var(self.trLabel_r), ", std:",
                          np.std(self.trLabel_r), ", range:", (np.amin(self.trLabel_r),np.amax(self.trLabel_r)))
                elif k == 'trNumframes':
                    print("trNumframes.shape: ", v.shape)
                    self.trNumframes = np.transpose(np.array(v))
                    
                elif k == 'cvData':
                    print("cvData.shape: ", v.shape)
                    self.cvData = np.transpose(np.array(v))
                    print("cvData-> mean:", np.mean(self.cvData), ", var:", np.var(self.cvData), ", std:",
                          np.std(self.cvData), ", range:", (np.amin(self.cvData),np.amax(self.cvData)))
                elif k == 'cvLabel_i':
                    print("cvLabel_i.shape: ", v.shape)
                    self.cvLabel_i = np.transpose(np.array(v))
                    print("cvLabel_i-> mean:", np.mean(self.cvLabel_i), ", var:", np.var(self.cvLabel_i), ", std:",
                          np.std(self.cvLabel_i), ", range:", (np.amin(self.cvLabel_i),np.amax(self.cvLabel_i)))
                elif k == 'cvLabel_r':
                    print("cvLabel_r.shape: ", v.shape)
                    self.cvLabel_r = np.transpose(np.array(v))
                    print("cvLabel_r-> mean:", np.mean(self.cvLabel_r), ", var:", np.var(self.cvLabel_r), ", std:",
                          np.std(self.cvLabel_r), ", range:", (np.amin(self.cvLabel_r),np.amax(self.cvLabel_r)))
                elif k == 'cvNumframes':
                    print("cvNumframes.shape: ", v.shape)
                    self.cvNumframes = np.transpose(np.array(v))

            self.trLabel = np.concatenate((self.trLabel_r, self.trLabel_i), axis=1)
            self.cvLabel = np.concatenate((self.cvLabel_r, self.cvLabel_i), axis=1)


    def ready_batchID(self, total_num_samples, batch_size):
        # TRAIN: total_num_samples = self.trData.shape[0] = 195192
        # DEV: total_num_samples = self.trData.shape[0] = 44961

        batchID = []
        num_batch = math.ceil(total_num_samples/batch_size)

        for b in range( int(num_batch) ):
            s = b*batch_size
            e = (b+1)*batch_size -1

            if e >= total_num_samples:
                e = total_num_samples - 1

            batchID.append((s,e))

        return np.array(batchID,ndmin=2)


    def suffle_data(self, total_num_samples):
        # TRAIN: total_num_samples = self.trData.shape[0] = 195192
        # DEV: total_num_samples = self.trData.shape[0] = 44961

        return  np.random.permutation(total_num_samples)


    def next_batch(self, total_num_samples, batch_size, isTrainCycle=True):
        # TRAIN: total_num_samples = self.trData.shape[0] = 195192
        # DEV: total_num_samples = self.trData.shape[0] = 44961

        while True:
            batchID = self.ready_batchID(total_num_samples, batch_size) 
            seq = self.suffle_data(total_num_samples)

            for batch in range(batchID.shape[0]):
                if isTrainCycle:
                    x = opts.trData[ seq[batchID[batch][0]:batchID[batch][1] ] ]
                    y = opts.trLabel[ seq[batchID[batch][0]:batchID[batch][1] ] ]

                else:
                    x = opts.cvData[seq[batchID[batch][0]:batchID[batch][1]]]
                    y = opts.cvLabel[seq[batchID[batch][0]:batchID[batch][1]]]

                # print('Next Batch', x.shape, y.shape)
                yield [x, y]


    def next_batch2(self, batch_size, isTrainCycle=True):
        if isTrainCycle:
            # selected_indics = np.random.choice(10, size=batch_size, replace=False)
            selected_indics = np.random.randint(195192, size=batch_size)
        else:
            # selected_indics = np.random.randint(7, size= batch_size)
            selected_indics = np.random.randint(44961, size=batch_size)

        # print ("selected_indics: ",selected_indics)
        if isTrainCycle:
            x = opts.trData[selected_indics]
            y = np.concatenate((opts.trLabel_r, opts.trLabel_i), axis=1)[selected_indics]

        else:
            x = opts.cvData[selected_indics]
            y = np.concatenate((opts.cvLabel_r, opts.cvLabel_i), axis=1)[selected_indics]

        # print('Next Batch', x.shape, y.shape)
        return [x, y]

In [5]:
opts = Opts(DNN_MODEL_FILE, DNN_DATA_FILE)

Opt key: ['#refs#', 'opts']
key: ARMA_order
key: ada_grad_eps
key: ada_sgd_scale
key: change_momentum_point
key: cost_function
key: cv_interval
key: dim_input
key: dim_output
key: drop_ratio
key: eval_on_gpu
key: final_momentum
key: hid_struct
key: initial_momentum
key: isDropout
key: isDropoutInput
key: isGPU
key: isNormalize
key: isPretrain
key: learner
key: net_struct
Opts Net Stuct: <HDF5 object reference>
Opts Net Stuct: <HDF5 object reference>
Opts Net Stuct: <HDF5 object reference>
Opts Net Stuct: <HDF5 object reference>
Opts Net Stuct: <HDF5 object reference>
key: rbm_batch_size
self.rbm_batch_size: 1024
key: rbm_learn_rate_binary
key: rbm_learn_rate_real
key: rbm_max_epoch
key: save_on_fly
key: sgd_batch_size
self.sgd_batch_size: 1024
key: sgd_learn_rate
key: sgd_max_epoch
key: split_tanh1_c1
key: split_tanh1_c2
key: tr_mu
key: tr_std
key: unit_type_hidden
key: unit_type_output
Opts h5py keys: ['cvData', 'cvLabel_i', 'cvLabel_r', 'cvNumframes', 'trData', 'trLabel_i', 'trLabel_

### Parameters

In [6]:
## HAVE to use OPS PARAMS ***********
learning_rate = 0.001
# training_epochs = 80
# batch_size = 256
display_step = 1

### Network Parameters

### Writing in file

In [7]:
def write_file(best_weights, best_biases, DNN_NET_FILE):
    W_1, W_2, W_3, W_4 = [np.array(best_weights['h1'], ndmin=2), np.array(best_weights['h2'], ndmin=2),
                          np.array(best_weights['h3'], ndmin=2), np.array([], ndmin=2)]
    W_1, W_2, W_3, W_4 = W_1.T, W_2.T, W_3.T, W_4

    print('W_1: ', W_1.shape, 'W_2: ', W_2.shape, 'W_3: ', W_3.shape, 'W_4: ', W_4.shape)

    b_1, b_2, b_3, b_4 = [np.array(best_biases['b1'], ndmin=2), np.array(best_biases['b2'], ndmin=2),
                          np.array(best_biases['b3'], ndmin=2), np.array([], ndmin=2)]
    b_1, b_2, b_3, b_4 = b_1.T, b_2.T, b_3.T, b_4

    print('b_1: ', b_1.shape, 'b_2: ', b_2.shape, 'b_3: ', b_3.shape, 'b_4: ', b_4.shape)

    Wo, bo = np.array(best_weights['out'], ndmin=2), np.array(best_biases['out'], ndmin=2)

    Wo1_1, Wo1_2, Wo1_3, Wo1_4 = [np.array([], ndmin=2), np.array([], ndmin=2), np.array([], ndmin=2), Wo[:, :963].T]
    bo1_1, bo1_2, bo1_3, bo1_4 = [np.array([], ndmin=2), np.array([], ndmin=2), np.array([], ndmin=2),
                                  bo[:, :963].T]
    # Wo1_1, Wo1_2, Wo1_3, Wo1_4 = [np.array([], ndmin=2), np.array([], ndmin=2), np.array([], ndmin=2), Wo[:963]]
    # bo1_1, bo1_2, bo1_3, bo1_4 = [np.array([],ndmin=2), np.array([],ndmin=2), np.array([],ndmin=2),
    #                               np.reshape(np.transpose(bo[:963]), (963, 1))]


    Wo2_1, Wo2_2, Wo2_3, Wo2_4 = [np.array([], ndmin=2), np.array([], ndmin=2), np.array([], ndmin=2), Wo[:, 963:].T]
    bo2_1, bo2_2, bo2_3, bo2_4 = [np.array([], ndmin=2), np.array([], ndmin=2), np.array([], ndmin=2),
                                  bo[:, 963:].T]
    # Wo2_1, Wo2_2, Wo2_3, Wo2_4 = [np.array([],ndmin=2), np.array([],ndmin=2), np.array([],ndmin=2), Wo[963:]]
    # bo2_1, bo2_2, bo2_3, bo2_4 = [np.array([],ndmin=2), np.array([],ndmin=2), np.array([],ndmin=2),
    #                               np.reshape(np.transpose(bo[963:]), (963, 1))]

    print('Wo1_1: ', Wo1_1.shape, 'Wo1_2: ', Wo1_2.shape, 'Wo1_3: ', Wo1_3.shape, 'Wo1_4: ', Wo1_4.shape)
    print('bo1_1: ', bo1_1.shape, 'bo1_2: ', bo1_2.shape, 'bo1_3: ', bo1_3.shape, 'bo1_4: ', bo1_4.shape)

    print('Wo2_1: ', Wo2_1.shape, 'Wo2_2: ', Wo2_2.shape, 'Wo2_3: ', Wo2_3.shape, 'Wo2_4: ', Wo2_4.shape)
    print('bo2_1: ', bo2_1.shape, 'bo2_2: ', bo2_2.shape, 'bo2_3: ', bo2_3.shape, 'bo2_4: ', bo2_4.shape)

    # Param_Dict = {'W': np.transpose([W_1,W_2, W_3]), 'b':np.transpose([b_1,b_2, b_3]) }
    # Param_Dict = {'W': { (W_1, W_2, (W_3)}, 'b': {(b_1), (b_2), (b_3)}}
    Param_Dict = np.core.records.fromarrays(
        [[W_1, W_2, W_3, W_4], [b_1, b_2, b_3, b_4], [Wo1_1, Wo1_2, Wo1_3, Wo1_4], [bo1_1, bo1_2, bo1_3, bo1_4],
         [Wo2_1, Wo2_2, Wo2_3, Wo2_4], [bo2_1, bo2_2, bo2_3, bo2_4]], names='W,b,Wo1,bo1,Wo2,bo2')

    # print(Param_Dict.shape, Param_Dict)
    master_dict = {'struct_net': [Param_Dict]}

    sio.savemat(DNN_NET_FILE, master_dict, format='5', long_field_names=True)

### Custom loss function

In [8]:
def customLoss(yTrue,yPred):
    print('LOSS-fun:',type(yTrue), yTrue.get_shape()[0].value)
    
    # Define loss and optimizer
#     r, c = yTrue.shape
    c = 963 + 963
    r = 128 # opts.sgd_batch_size

    cost1 = K.sum(K.square(yTrue[:, :c//2]- yPred[:, :c//2]))
    cost2 = K.sum(K.square(yTrue[:, c//2:]- yPred[:, c//2:]))
    return 0.5*(cost1+cost2)/r
    

### DNN variables

In [9]:
batch_size = 128 #opts.sgd_batch_size
epochs = 30

n_input_sz = 1951920
n_out_sz = 449610

n_input = 1230  # data input
n_hidden_1 = 2048  # 1st layer number of neurons
n_hidden_2 = 2048  # 2nd layer number of neurons
n_hidden_3 = 2048  # 3rd layer number of neurons
n_classes = (963 + 963)


### Pre-trained DNN

In [10]:
model = Sequential()

model.add(Dense(n_hidden_1, activation='relu', input_shape=(n_input,)))
# we will add 2 more layers
model.add(Dense(n_classes, activation='linear'))

# Set callback functions to early stop training and save the best model so far
callbacks = [EarlyStopping(monitor='val_acc', patience=5, min_delta=1e-6, verbose=1, mode='auto'), 
             ModelCheckpoint(filepath= MODEL_FILE, monitor='val_acc', save_best_only=True)]
          
model.compile(loss = customLoss, optimizer = 'adam', metrics = ['accuracy','mse', 'mae', 'mape', 'cosine'])
print(model.summary())

history = model.fit_generator(opts.next_batch(opts.trData.shape[0], batch_size),  
                    validation_data=opts.next_batch(opts.cvData.shape[0], batch_size, isTrainCycle=False),
                    epochs=epochs, steps_per_epoch=int(math.ceil(n_input_sz/batch_size)),
                    validation_steps=int(math.ceil(n_out_sz/batch_size)), 
                    verbose=1, callbacks=callbacks)

model.save(SAVE_MODEL_FILE)

LOSS-fun: <class 'tensorflow.python.framework.ops.Tensor'> None
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 2048)              2521088   
_________________________________________________________________
dense_2 (Dense)              (None, 1926)              3946374   
Total params: 6,467,462
Trainable params: 6,467,462
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/30
15250/15250 [==============================] - 448s - loss: 15.9496 - acc: 0.0284 - mean_squared_error: 0.0167 - mean_absolute_error: 0.0459 - mean_absolute_percentage_error: 20734.2915 - cosine_proximity: -1.1466e-04 - val_loss: 15.6641 - val_acc: 0.0276 - val_mean_squared_error: 0.0164 - val_mean_absolute_error: 0.0465 - val_mean_absolute_percentage_error: 16721.4058 - val_cosine_proximity: -1.0705e-04
Epoch 2/30
15250/15250 [================

### Add 2nd hidden layer

In [11]:
SAVE_MODEL_FILE_2 = "./dnn_models/py_model_2_"+ Code_VERSION+".h5"
model = load_model(SAVE_MODEL_FILE, custom_objects={'customLoss':customLoss})

print('[OLD] Number of layers (except input layer, including output layer)->\n', model.layers)

model.pop()
model.add(Dense(n_hidden_2, activation='relu'))
# we will add 1 more layers
model.add(Dense(n_classes, activation='linear'))

print('[NEW] Number of layers (except input layer, including output layer)->\n', model.layers)




# Set callback functions to early stop training and save the best model so far
callbacks = [EarlyStopping(monitor='val_acc', patience=5, min_delta=1e-6, verbose=1, mode='auto'), 
             ModelCheckpoint(filepath= MODEL_FILE, monitor='val_acc', save_best_only=True)]
          
model.compile(loss = customLoss, optimizer = 'adam', metrics = ['accuracy','mse', 'mae', 'mape', 'cosine'])
print(model.summary())

history = model.fit_generator(opts.next_batch(opts.trData.shape[0], batch_size),  
                    validation_data=opts.next_batch(opts.cvData.shape[0], batch_size, isTrainCycle=False),
                    epochs=epochs, steps_per_epoch=int(math.ceil(n_input_sz/batch_size)),
                    validation_steps=int(math.ceil(n_out_sz/batch_size)), 
                    verbose=1, callbacks=callbacks)

model.save(SAVE_MODEL_FILE_2)

LOSS-fun: <class 'tensorflow.python.framework.ops.Tensor'> None
[OLD] Number of layers (except input layer, including output layer)->
 [<keras.layers.core.Dense object at 0x7f55d0834cc0>, <keras.layers.core.Dense object at 0x7f55d0834f60>]
[NEW] Number of layers (except input layer, including output layer)->
 [<keras.layers.core.Dense object at 0x7f55d0834cc0>, <keras.layers.core.Dense object at 0x7f55cf84c518>, <keras.layers.core.Dense object at 0x7f55cc1c54e0>]
LOSS-fun: <class 'tensorflow.python.framework.ops.Tensor'> None
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 2048)              2521088   
_________________________________________________________________
dense_3 (Dense)              (None, 2048)              4196352   
_________________________________________________________________
dense_4 (Dense)              (None, 1926)              3946374   
Total 

In [13]:
SAVE_MODEL_FILE_3 = "./dnn_models/py_model_3_"+ Code_VERSION+".h5"
model = load_model(SAVE_MODEL_FILE_2, custom_objects={'customLoss':customLoss})

print('[OLD] Number of layers (except input layer, including output layer)->\n', model.layers)

model.pop()
model.add(Dense(n_hidden_2, activation='relu'))
model.add(Dense(n_classes, activation='linear'))

print('[NEW] Number of layers (except input layer, including output layer)->\n', model.layers)




# Set callback functions to early stop training and save the best model so far
callbacks = [EarlyStopping(monitor='val_acc', patience=5, min_delta=1e-6, verbose=1, mode='auto'), 
             ModelCheckpoint(filepath= MODEL_FILE, monitor='val_acc', save_best_only=True)]
          
model.compile(loss = customLoss, optimizer = 'adam', metrics = ['accuracy','mse', 'mae', 'mape', 'cosine'])
print(model.summary())

history = model.fit_generator(opts.next_batch(opts.trData.shape[0], batch_size),  
                    validation_data=opts.next_batch(opts.cvData.shape[0], batch_size, isTrainCycle=False),
                    epochs=epochs, steps_per_epoch=int(math.ceil(n_input_sz/batch_size)),
                    validation_steps=int(math.ceil(n_out_sz/batch_size)), 
                    verbose=1, callbacks=callbacks)

model.save(SAVE_MODEL_FILE_3)

LOSS-fun: <class 'tensorflow.python.framework.ops.Tensor'> None
[OLD] Number of layers (except input layer, including output layer)->
 [<keras.layers.core.Dense object at 0x7f52e46ac278>, <keras.layers.core.Dense object at 0x7f52e46ac4e0>, <keras.layers.core.Dense object at 0x7f52e47beb00>]
[NEW] Number of layers (except input layer, including output layer)->
 [<keras.layers.core.Dense object at 0x7f52e46ac278>, <keras.layers.core.Dense object at 0x7f52e46ac4e0>, <keras.layers.core.Dense object at 0x7f52cc756eb8>, <keras.layers.core.Dense object at 0x7f52cc756518>]
LOSS-fun: <class 'tensorflow.python.framework.ops.Tensor'> None
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 2048)              2521088   
_________________________________________________________________
dense_3 (Dense)              (None, 2048)              4196352   
__________________________________